In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Initialise Model
m = gp.Model("7-4-2")


# Initialise Decision Variables using a list of indices from 1 to 420
indices = list(range(0, 420))

beta_size = 10  # B0, ... ,B9
indices_2 = list(range(0, beta_size))

x = m.addVars(indices, vtype=GRB.BINARY, name='x')
c = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='c') 
z = m.addVars(indices, vtype=GRB.INTEGER, name='z')
y = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='y')
beta = m.addVars(indices_2, lb = 0, vtype=GRB.INTEGER, name='Beta')


# Set Objective function
m.setObjective(y.sum())


# Define the alpha matrix
import itertools
# Create all unique pairs for the face-up cards (left and right)
card_values = range(1, 8)  # Card values from 1 to 7
face_up_pairs = list(itertools.permutations(card_values, 2))

# Generate alpha matrix
alpha_matrix = []
condensed_matrix = []


for left, right in face_up_pairs:
    for hidden1, hidden2 in itertools.combinations(card_values, 2):
        if hidden1 not in (left, right) and hidden2 not in (left, right): 
            alpha_i1 = left
            alpha_i2 = right  
            alpha_i3 = 1 if left > right else 0
            alpha_i4 = 1 if hidden1 > left else 0
            alpha_i5 = 1 if hidden1 > right else 0
            alpha_j6 = hidden1
            alpha_i7 = 1 if hidden2 > left else 0
            alpha_i8 = 1 if hidden2 > right else 0
            alpha_j9 = hidden2
            
            alpha_matrix.append([alpha_i1, alpha_i2, alpha_i3, alpha_i4, alpha_i5, alpha_j6, alpha_i7, alpha_i8, alpha_j9])   # j6 and j9 are the values of hidden 1 and hidden 2 respectively

            sorted_row = sorted([alpha_i1, alpha_i2, alpha_j6, alpha_j9])
            condensed_matrix.append(sorted_row)

# alpha_matrix
# condensed_matrix

# This is to for ease of coding out the matching constraints (see the variable 'new_matrix')
from collections import defaultdict

# Initialize a dictionary to store the rows grouped by their elements
grouped_rows = defaultdict(list)

# Iterate through the condensed matrix
for index, row in enumerate(condensed_matrix):
    # Convert the row to a tuple to make it hashable and usable as a dictionary key
    key = tuple(row)
    # Append the row index to the list corresponding to the key
    grouped_rows[key].append(index)

# Define the size of the new matrix
num_rows = len(grouped_rows)
num_cols = 12  # Assuming all rows have the same number of columns

# Initialize a new matrix with zeros
new_matrix = [[0] * num_cols for _ in range(num_rows)]

# Iterate through the grouped rows
for row_index, indices in enumerate(grouped_rows.values()):
    # Fill the new matrix with indices
    for col_index, index in enumerate(indices):
        new_matrix[row_index][col_index] = index

# Print the new matrix
for row in new_matrix:
    print(row)


# Create constraints
P = 13 

for i in range(420):
    m.addConstr(y[i] + (1 - x[i]) * P >= c[i])
    m.addConstr(c[i] >=  (1 - x[i]))
    m.addConstr(c[i] <=  P - 1)    
    
for i in range(35):
    m.addConstr(gp.quicksum(x[new_matrix[i][j]] for j in range(12)) == 1)

for i in range(42):
    m.addConstr(gp.quicksum(x[10*i + j] for j in range(10)) <= 1, 'x_sum_{}'.format(i))

for i in range(420):
    m.addConstr(c[i] == beta[0] + gp.quicksum(beta[k+1] * alpha_matrix[i][k] for k in range(0, 9)) - z[i] * P)


# Additional Cuts
m.addConstr(beta.sum() >= 1)

for i in range(10):
    m.addConstr(beta[i] <= P-1)  # P = 13 for (7-4-2) model

m.addConstr(beta[6] >= 1)
m.addConstr(beta[9] >= 1)


# Solve the Model
m.setParam('TimeLimit', 10*60) # Set a time limit of 30 minutes
m.optimize()



# Print out the coefficients
for i in range(10):
    print('beta[{}]: {}'.format(i, beta[i].X))

# Print the values of the 'c' variables
# for i in range(420):
#    print('c[{}]: {}'.format(i, c[i].X))



# Initialize a counter for zero
zero_count = 0

# Print the cost values of the matching links
for i in range(420):
    # print('x[{}]: {}'.format(i, x[i].X))
    if (x[i].X == 1):
        print('x[{}]: {}'.format(i, x[i].X), 'c[{}]: {}'.format(i, c[i].X), alpha_matrix[i])
        if c[i].X == 0.0:
            zero_count += 1

print(zero_count)

